# Building a database for crime reports

This project is based on creating a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file. I will be creating the table inside a schema named crimes. I will also create the readonly and readwrite groups with the appropriate privileges. Finally, I will also need to create one user for each of these groups.

### Creating the Crime Database

Connect to the dq database with user dq
Start by creating a database for storing the crime data as well as a schema for containing the tables.

Keep in mind that in order to create a database, you need to set the connection.autocommit to True

In [1]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()


In [2]:
conn = psycopg2.connect(dbname="crime_db", user="dq")

To create a schema named crimes

In [3]:
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

### Obtaining the Column Names and Sample

Gather some data about the crime dataset so that you can easily select the right datatypes to use in the table.

In [4]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
    

### Creating an Auxiliary Function

Before creating a table to store the crime data, identify the proper datatypes for the columns. 

This function will be useful for two reasons:

Checking whether an enumerated datatype might be a good choice for representing a column

Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

In [5]:

def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


### Finding the Maximum Length

Another important aspect is to know the longest word in any column containing textual data.



In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


To compute the maximum length of any value in the description column.

In [7]:
descriptions = get_col_set("boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


### Creating the Table

Create a table named boston_crimes inside the crimes schema of the crime_db database. 

In [8]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [9]:
# Create the enumerated datatype for representing the weekday.
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# Create the table.
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

### Loading the Data

In [10]:
# Load the data from boston.csv into the table boston_crimes that is in the crimes schema.
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# Print the number of rows to ensure that they were loaded.
print(len(cur.fetchall()))

298329


### Revoking Public Privileges

By following the least privilege principle, make sure that there are no privileges inherited from the public group and on the public schema.

In [12]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### Creating User Groups

create user groups

The readonly group is supposed to only have privileges to perform SELECT queries. 

In contrast, the readwrite group should be able to perform SELECT, INSERT, DELETE, and UPDATE queries.

ENSURE: 

Connection privileges on the crime_db — otherwise, they won't be able to do anything. 

Usage of the crimes schema — this is where all data will be stored.

In [13]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [14]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE,UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

### Creating Users

creating one user in each group

In [15]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

### Test the Database Setup

In [16]:
# Close the old connection to test with a brand new connection.
conn.close()

In [17]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# Check users and groups.
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

